In [0]:
data =[("James ","","Smith","36636","M",3000),
              ("Michael ","Rose","","40288","M",4000),
              ("Robert ","","Williams","42114","M",4000),
              ("Maria ","Anne","Jones","39192","F",4000),
              ("Jen","Mary","Brown","","F",-1)]
columns=["firstname","middlename","lastname","dob","gender","salary"]
df=spark.createDataFrame(data,columns)
df.printSchema()
df.show(truncate=False)

root
-- firstname: string (nullable = true)
-- middlename: string (nullable = true)
-- lastname: string (nullable = true)
-- dob: string (nullable = true)
-- gender: string (nullable = true)
-- salary: long (nullable = true)

+---------+----------+--------+-----+------+------+
firstname|middlename|lastname|dob |gender|salary|
+---------+----------+--------+-----+------+------+
James | |Smith |36636|M |3000 |
Michael |Rose | |40288|M |4000 |
Robert | |Williams|42114|M |4000 |
Maria |Anne |Jones |39192|F |4000 |
Jen |Mary |Brown | |F |-1 |
+---------+----------+--------+-----+------+------+

In [0]:
df.write.parquet("/tmp/output/People.parquet")

In [0]:
parDF=spark.read.parquet("/tmp/output/People.parquet")


In [0]:
df.write.mode('append').parquet("/tmp/output/people.parquet")

In [0]:
df.write.mode('overwrite').parquet("/tmp/output/people.parquet")

In [0]:
df.createOrReplaceTempView("ParquetTable")
spark.sql("select * from ParquetTable where salary >= 4000 ").show()

+---------+----------+--------+-----+------+------+
firstname|middlename|lastname| dob|gender|salary|
+---------+----------+--------+-----+------+------+
 Michael | Rose| |40288| M| 4000|
 Robert | |Williams|42114| M| 4000|
 Maria | Anne| Jones|39192| F| 4000|
+---------+----------+--------+-----+------+------+

In [0]:
spark.sql("CREATE TEMPORARY VIEW PERSON USING parquet OPTIONS (path \"/tmp/output/people.parquet\")")
spark.sql("SELECT * FROM PERSON").show()

+---------+----------+--------+-----+------+------+
firstname|middlename|lastname| dob|gender|salary|
+---------+----------+--------+-----+------+------+
 Maria | Anne| Jones|39192| F| 4000|
 Robert | |Williams|42114| M| 4000|
 Michael | Rose| |40288| M| 4000|
 James | | Smith|36636| M| 3000|
 Jen| Mary| Brown| | F| -1|
+---------+----------+--------+-----+------+------+

In [0]:
df.write.partitionBy("gender","salary").mode("overwrite").parquet("/tmp/output/people2.parquet")

In [0]:
parDF2=spark.read.parquet("/tmp/output/people2.parquet/gender=M")
parDF2.show(truncate=False)

+---------+----------+--------+-----+------+
firstname|middlename|lastname|dob |salary|
+---------+----------+--------+-----+------+
Robert | |Williams|42114|4000 |
Michael |Rose | |40288|4000 |
James | |Smith |36636|3000 |
+---------+----------+--------+-----+------+

In [0]:
spark.sql("CREATE TEMPORARY VIEW PERSON2 USING parquet OPTIONS (path \"/tmp/output/people2.parquet/gender=F\")")
spark.sql("SELECT * FROM PERSON2" ).show()

+---------+----------+--------+-----+------+
firstname|middlename|lastname| dob|salary|
+---------+----------+--------+-----+------+
 Maria | Anne| Jones|39192| 4000|
 Jen| Mary| Brown| | -1|
+---------+----------+--------+-----+------+